In [62]:
import pandas as pd
import numpy as np
import os
path = 'C:/Users/dubrangala/OneDrive - VMware, Inc/Case Studies/hackerarth_prediction/dataset'
os.chdir(path)

In [63]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [64]:
## Read Master Data
final_data_master = pd.read_csv("final_data_master.csv")

In [65]:
def summary_stat (df):
    missing_cnt = df.isnull().sum()
    missing_prop = round(100*df.isnull().sum()/len(df),2)
    uniqv = df.apply(lambda x: x.nunique())
    ctype = df.apply(lambda x: x.dtype)
    mydf = pd.DataFrame({'Column Type':ctype, 'Number of Unique':uniqv, 'Total missing':missing_cnt, 'missing percentage':missing_prop})
    return(mydf)

In [66]:
mydata_dictinary = summary_stat(final_data_master)
mydata_dictinary = mydata_dictinary.reset_index()
mydata_dictinary.head()

,index,Column Type,Number of Unique,Total missing,missing percentage
0,patient_id,object,31270,0,0.0
1,patient_age,float64,1423,0,0.0
2,mothers_age,float64,5977,0,0.0
3,fathers_age,float64,5911,0,0.0
4,no._of_previous_abortion,float64,4218,0,0.0


In [67]:
final_data_master.myflag.value_counts()

train    21805
test      9465
Name: myflag, dtype: int64

In [68]:
final_data_master.columns

Index(['patient_id', 'patient_age', 'mothers_age', 'fathers_age',
       'no._of_previous_abortion',
       'white_blood_cell_count_thousand_per_microliter',
       'inherited_from_father', 'maternal_gene', 'respiratory_rate_breathsmin',
       'heart_rate_ratesmin', 'gender', 'birth_asphyxia',
       'autopsy_shows_birth_defect_if_applicable', 'place_of_birth',
       'folic_acid_details_periconceptional', 'ho_serious_maternal_illness',
       'ho_radiation_exposure_xray', 'ho_substance_abuse',
       'assisted_conception_ivfart',
       'history_of_anomalies_in_previous_pregnancies', 'birth_defects',
       'blood_test_result', 'blood_cell_count_mcl', 'test_1', 'test_2',
       'test_3', 'test_4', 'test_5', 'symptom_1', 'symptom_2', 'symptom_3',
       'symptom_4', 'symptom_5', 'genes_in_mothers_side', 'paternal_gene',
       'status', 'parental_consent', 'followup', 'myflag', 'genetic_disorder',
       'disorder_subclass', 'genetic_disorder_old', 'disorder_subclass_v1',
       'gene

In [69]:
final_data_master[final_data_master.myflag=='train'].genetic_disorder_v1.value_counts()

Mitochondrial genetic inheritance disorders     11174
Single-gene inheritance diseases                 8371
Multifactorial genetic inheritance disorders     2260
Name: genetic_disorder_v1, dtype: int64

In [70]:
final_data_master[final_data_master.myflag=='train'].disorder_subclass.value_counts()

Leigh syndrome                         5160
Mitochondrial myopathy                 4405
Cystic fibrosis                        3448
Tay-Sachs                              2833
Diabetes                               1817
Hemochromatosis                        1355
Leber's hereditary optic neuropathy     648
Alzheimer's                             152
Cancer                                   97
Name: disorder_subclass, dtype: int64

In [71]:
final_data_master['mytarget'] = final_data_master['genetic_disorder_v1']+str("::")+final_data_master['disorder_subclass']
final_data_master[final_data_master.myflag=='train'].mytarget.value_counts()

Mitochondrial genetic inheritance disorders::Leigh syndrome                         5160
Mitochondrial genetic inheritance disorders::Mitochondrial myopathy                 4405
Single-gene inheritance diseases::Cystic fibrosis                                   3448
Single-gene inheritance diseases::Tay-Sachs                                         2833
Multifactorial genetic inheritance disorders::Diabetes                              1817
Single-gene inheritance diseases::Hemochromatosis                                   1355
Mitochondrial genetic inheritance disorders::Leber's hereditary optic neuropathy     648
Multifactorial genetic inheritance disorders::Alzheimer's                            152
Multifactorial genetic inheritance disorders::Cancer                                  97
Name: mytarget, dtype: int64

In [72]:
df_train = final_data_master[final_data_master.myflag=='train'].copy()
df_valid = final_data_master[final_data_master.myflag!='train'].copy()
df_train = df_train[~df_train['disorder_subclass'].isnull()].copy()

In [73]:
df_train.to_csv("df_train_checking.csv", index=False)

### Model coding - Functions

In [16]:
#import libraries
import lightgbm as lgbm
import pandas as pd
import numpy as np
import os
import datetime as dt
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
randSeed=17

In [18]:
from sklearn.ensemble import IsolationForest
from pyod.models.hbos import HBOS
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.knn import KNN
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.lof import LOF
from pyod.models.vae import VAE
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.iforest import IForest
import tensorflow as tf
from collections import Counter

In [19]:
Counter(df_train.mytarget)

Counter({"Mitochondrial genetic inheritance disorders::Leber's hereditary optic neuropathy": 648,
         'Single-gene inheritance diseases::Cystic fibrosis': 3448,
         'Multifactorial genetic inheritance disorders::Diabetes': 1817,
         'Mitochondrial genetic inheritance disorders::Leigh syndrome': 5160,
         'Multifactorial genetic inheritance disorders::Cancer': 97,
         'Single-gene inheritance diseases::Tay-Sachs': 2833,
         'Single-gene inheritance diseases::Hemochromatosis': 1355,
         'Mitochondrial genetic inheritance disorders::Mitochondrial myopathy': 4405,
         nan: 1890,
         "Multifactorial genetic inheritance disorders::Alzheimer's": 152})

In [21]:
class_encoding = {"Mitochondrial genetic inheritance disorders::Leber's hereditary optic neuropathy": 0,
         'Single-gene inheritance diseases::Cystic fibrosis': 1,
         'Multifactorial genetic inheritance disorders::Diabetes': 2,
         'Mitochondrial genetic inheritance disorders::Leigh syndrome': 3,
         'Multifactorial genetic inheritance disorders::Cancer': 4,
         'Single-gene inheritance diseases::Tay-Sachs': 5,
         'Single-gene inheritance diseases::Hemochromatosis': 6,
         'Mitochondrial genetic inheritance disorders::Mitochondrial myopathy': 7,
         "Multifactorial genetic inheritance disorders::Alzheimer's": 8}

class_decoding = {0:"Mitochondrial genetic inheritance disorders::Leber's hereditary optic neuropathy",
         1:'Single-gene inheritance diseases::Cystic fibrosis',
         2:'Multifactorial genetic inheritance disorders::Diabetes',
         3:'Mitochondrial genetic inheritance disorders::Leigh syndrome',
         4:'Multifactorial genetic inheritance disorders::Cancer',
         5:'Single-gene inheritance diseases::Tay-Sachs',
         6:'Single-gene inheritance diseases::Hemochromatosis',
         7:'Mitochondrial genetic inheritance disorders::Mitochondrial myopathy',
         8:"Multifactorial genetic inheritance disorders::Alzheimer's"}

In [24]:
df_train['target'] = df_train['mytarget'].apply(lambda x:class_encoding[x])
df_valid['target'] = 99
df_train.target.value_counts()

3    5160
7    4405
1    3448
5    2833
2    1817
6    1355
0     648
8     152
4      97
Name: target, dtype: int64

In [25]:
# Anomaly scoring methods-unsupervised
def get_IF_scores (dataframe,dataframeNms):
    clf=IsolationForest(n_estimators=200, max_samples='auto', contamination=0.35,
                        bootstrap=True, n_jobs=-1, random_state=randSeed, verbose=1)
    clf.fit(dataframe[dataframeNms])
    anomalyScore = 1-clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_HBOS_scores (dataframe,dataframeNms):
    clf=HBOS(n_bins =200,contamination=0.3)
    clf.fit(dataframe[dataframeNms])
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_OCSVM_scores (dataframe,dataframeNms):
    clf=OCSVM(kernel='rbf', contamination=0.35,cache_size=2400)
    clf.fit(dataframe[dataframeNms])
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_PCA_scores (dataframe,dataframeNms):
    clf=PCA(n_components =10,  contamination=0.25,random_state=randSeed,standardization=False)  
    scaler= MinMaxScaler()
    df_temp = dataframe[dataframeNms]
    df_temp[df_temp.select_dtypes(include=[float]).columns.tolist()] = scaler.fit_transform(df_temp.select_dtypes(include=[float]))

    clf.fit(df_temp)

    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_KNN_scores (dataframe,dataframeNms):
    clf=KNN(n_neighbors=5, contamination=0.5,n_jobs=-1)
    clf.fit(dataframe[dataframeNms])
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_ABOD_scores (dataframe,dataframeNms):
    clf=ABOD(contamination=0.5,n_neighbors=5, method='fast')   
    scaler= MinMaxScaler()
    df_temp = dataframe[dataframeNms]
    df_temp[df_temp.select_dtypes(include=[float]).columns.tolist()] = scaler.fit_transform(df_temp.select_dtypes(include=[float]))

    clf.fit(df_temp)
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_CBLOF_scores (dataframe,dataframeNms):
    clf=CBLOF(n_clusters =15, contamination=0.5,random_state=randSeed, n_jobs=-1)   
    
    scaler= MinMaxScaler()
    df_temp = dataframe[dataframeNms]
    df_temp[df_temp.select_dtypes(include=[float]).columns.tolist()] = scaler.fit_transform(df_temp.select_dtypes(include=[float]))

    clf.fit(df_temp)
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_LOF_scores (dataframe,dataframeNms):
    clf=LOF(n_neighbors=8, contamination=0.45,n_jobs=-1)   
    scaler= MinMaxScaler()
    df_temp = dataframe[dataframeNms]
    df_temp[df_temp.select_dtypes(include=[float]).columns.tolist()] = scaler.fit_transform(df_temp.select_dtypes(include=[float]))

    clf.fit(df_temp)
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_VAE_scores (dataframe,dataframeNms):
    clf=VAE(encoder_neurons=[384,192, 96, 48], decoder_neurons=[48,96,192,384],contamination=0.3,batch_size=1024,
            epochs=10,random_state=randSeed)   
    scaler= MinMaxScaler()
    df_temp = dataframe[dataframeNms]
    df_temp[df_temp.select_dtypes(include=[float]).columns.tolist()] = scaler.fit_transform(df_temp.select_dtypes(include=[float]))

    clf.fit(df_temp.fillna(2))
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore


def get_FeatureBagging_HBOS_scores(dataframe,dataframeNms):
    clf=FeatureBagging(base_estimator=HBOS(n_bins =100,contamination=0.25), n_estimators=10, contamination=0.4,
                       n_jobs=-1,random_state=randSeed)   
    clf.fit(dataframe[dataframeNms])
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_FeatureBagging_PCA_scores(dataframe,dataframeNms):
    clf=FeatureBagging(base_estimator=PCA(n_components =20,  contamination=0.3,standardization=False), n_estimators=10, contamination=0.4,
                       n_jobs=-1,random_state=randSeed)   
    scaler= MinMaxScaler()
    df_temp = dataframe[dataframeNms]
    df_temp[df_temp.select_dtypes(include=[float]).columns.tolist()] = scaler.fit_transform(df_temp.select_dtypes(include=[float]))

    clf.fit(df_temp)
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

def get_FeatureBagging_IF_scores(dataframe,dataframeNms):
    clf=FeatureBagging(base_estimator=IForest(n_estimators=150, max_samples='auto', contamination=0.23,
                        bootstrap=True, n_jobs=-1,verbose=1), n_estimators=4, contamination=0.4,
                       n_jobs=-1,random_state=randSeed)   
    clf.fit(dataframe[dataframeNms])
    anomalyScore = clf.decision_function(dataframe[dataframeNms])
    return anomalyScore

In [26]:
def parameterSpace(selected_classfier='xgboost'):  
    
    # XGBOOST earlier exploration
    if(selected_classfier=='xgboost'):
        space = {
        'n_estimators': hp.quniform('n_estimators', 50, 800, 20),
        'max_depth':  hp.choice('max_depth', np.arange(5, 25, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 2, 30, 1),
        'subsample': hp.quniform('subsample', 0.1, 0.9, 0.005),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.09, 0.9, 0.05),
        'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 1.0, 0.1),
        'colsample_bynode': hp.quniform('colsample_bynode', 0.1, 1.0, 0.1),
        'learning_rate': hp.quniform('learning_rate', 0.015, 0.15, 0.005),
        'gamma':  hp.choice('gamma', np.arange(0, 3, dtype=int)),
        #'eval_metric': 'auc',
        'early_stopping_rounds':50,
        #'num_parallel_tree':50, # no need
        'nthread': 900,
        'updater':'grow_gpu_hist',
        #'n_jobs': 10, # gpu pred doesnot need n_jobs
        'silent': 1,
        'predictor':'gpu_predictor',
        'tree_method': 'gpu_hist',
        'verbosity':0,
        'missing':np.nan,
        'sampling_method':'gradient_based', # works only with gpu_hist
        'seed': randSeed
         }
    elif(selected_classfier=='lightgbm'):
        
        space = {
                       
            'num_leaves': hp.quniform('num_leaves', 20, 800, 10),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 0.95, 0.015),
            'learning_rate':hp.quniform('learning_rate', 0.015, 0.22, 0.005),
            'min_child_samples': hp.quniform('min_child_samples', 2, 100,2),
            'lambda_l1': hp.lognormal('lambda_l1', 1e-8, 3.0),
            'lambda_l2':  hp.lognormal('lambda_l2', 1e-8, 3.0),
            'bagging_freq': hp.quniform('bagging_freq', 2, 50,1),
            'subsample': hp.quniform('subsample', 0.1, 0.95, 0.015),
            'device': 'cpu',
            'first_metric_only':False,
            'n_jobs':-1,
            'eval_names': 'hackathon',
            'tree_learner': 'serial',
            'random_state': randSeed} 
        # 600 runs

    elif(selected_classfier=='RF'):
        space= { 
            'n_estimators': hp.quniform('n_estimators', 50, 1500, 20), #number of trees, change it to 1000 for better results
            'criterion':hp.choice('criterion', ['gini','entropy']),
            'max_depth':  hp.choice('max_depth', np.arange(2, 20, dtype=int)),
            'min_samples_split': hp.quniform('min_samples_split', 2, 200,2),
            'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5,1),
            'min_weight_fraction_leaf': hp.quniform('min_weight_fraction_leaf', 0.0, 0.48,0.01),
            'max_features':hp.choice('max_features',['auto', 'sqrt', 'log2']),
            'min_impurity_decrease': hp.quniform('min_impurity_decrease', 0.0, 0.9,0.01),
            'oob_score':hp.choice('oob_score',[False, True]),
            'max_samples': hp.quniform('max_samples', 0.01, 0.99,0.01)  
          
        }
       
    else:
        print('Only xgboost, lightgbm and randomforest are supported: RF,lightgbm,xgboost')
        
        
    return space

In [60]:
# merge into common dataframe for computing anomalies
df_train['flag_train'] = 1
df_valid['flag_train'] = 0
df = df_train.append(df_valid)
df.reset_index(drop=True, inplace=True)
del df_train
del df_valid
import gc
gc.collect()

93

#### Pre processing of data

In [28]:
exclude_list = ['genetic_disorder','disorder_subclass','genetic_disorder_old','disorder_subclass_v1','genetic_disorder_v1','myflag','patient_id','mytarget']
mydata_dictinary = mydata_dictinary[~mydata_dictinary['index'].isin(exclude_list)]
cat_var = mydata_dictinary[mydata_dictinary['Column Type']=='object']['index']
num_var = mydata_dictinary[mydata_dictinary['Column Type']!='object']['index']

In [29]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X_one_hot= enc.fit_transform(df[['inherited_from_father', 'maternal_gene', 'respiratory_rate_breathsmin', 'heart_rate_ratesmin', 'gender', 'birth_asphyxia', 'autopsy_shows_birth_defect_if_applicable', 'place_of_birth', 'folic_acid_details_periconceptional', 'ho_serious_maternal_illness', 'ho_radiation_exposure_xray', 'ho_substance_abuse', 'assisted_conception_ivfart', 'history_of_anomalies_in_previous_pregnancies', 'birth_defects', 'blood_test_result', 'genes_in_mothers_side', 'paternal_gene', 'status', 'parental_consent', 'followup']]).toarray()
enc.get_feature_names(['inherited_from_father', 'maternal_gene', 'respiratory_rate_breathsmin', 'heart_rate_ratesmin', 'gender', 'birth_asphyxia', 'autopsy_shows_birth_defect_if_applicable', 'place_of_birth', 'folic_acid_details_periconceptional', 'ho_serious_maternal_illness', 'ho_radiation_exposure_xray', 'ho_substance_abuse', 'assisted_conception_ivfart', 'history_of_anomalies_in_previous_pregnancies', 'birth_defects', 'blood_test_result', 'genes_in_mothers_side', 'paternal_gene', 'status', 'parental_consent', 'followup'])

array(['inherited_from_father_No', 'inherited_from_father_Yes',
       'maternal_gene_No', 'maternal_gene_Yes',
       'respiratory_rate_breathsmin_Normal (30-60)',
       'respiratory_rate_breathsmin_Tachypnea',
       'heart_rate_ratesmin_Normal', 'heart_rate_ratesmin_Tachycardia',
       'gender_Ambiguous', 'gender_Female', 'gender_Male',
       'birth_asphyxia_No', 'birth_asphyxia_Not available',
       'birth_asphyxia_Yes',
       'autopsy_shows_birth_defect_if_applicable_No',
       'autopsy_shows_birth_defect_if_applicable_Not applicable',
       'autopsy_shows_birth_defect_if_applicable_Yes',
       'place_of_birth_Home', 'place_of_birth_Institute',
       'place_of_birth_Not available',
       'folic_acid_details_periconceptional_No',
       'folic_acid_details_periconceptional_Yes',
       'ho_serious_maternal_illness_No',
       'ho_serious_maternal_illness_Yes', 'ho_radiation_exposure_xray_No',
       'ho_radiation_exposure_xray_Not applicable',
       'ho_radiation_exposur

In [30]:
X_one_hot = pd.DataFrame(X_one_hot,columns=enc.get_feature_names(['inherited_from_father', 'maternal_gene', 'respiratory_rate_breathsmin', 'heart_rate_ratesmin', 'gender', 'birth_asphyxia', 'autopsy_shows_birth_defect_if_applicable', 'place_of_birth', 'folic_acid_details_periconceptional', 'ho_serious_maternal_illness', 'ho_radiation_exposure_xray', 'ho_substance_abuse', 'assisted_conception_ivfart', 'history_of_anomalies_in_previous_pregnancies', 'birth_defects', 'blood_test_result', 'genes_in_mothers_side', 'paternal_gene', 'status', 'parental_consent', 'followup']).tolist() )
X_one_hot = X_one_hot.astype('bool')
df = pd.concat([df, X_one_hot], axis=1)
df['target'] = df['target'].fillna(-1)
df = df.drop(['inherited_from_father', 'maternal_gene', 'respiratory_rate_breathsmin', 'heart_rate_ratesmin', 'gender', 'birth_asphyxia', 'autopsy_shows_birth_defect_if_applicable', 'place_of_birth', 'folic_acid_details_periconceptional', 'ho_serious_maternal_illness', 'ho_radiation_exposure_xray', 'ho_substance_abuse', 'assisted_conception_ivfart', 'history_of_anomalies_in_previous_pregnancies', 'birth_defects', 'blood_test_result', 'genes_in_mothers_side', 'paternal_gene', 'status', 'parental_consent', 'followup'],axis=1)

In [31]:
corr_df = df.corr()
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool))
corNms = [column for column in upper.columns if any(upper[column].abs() > 0.99)]

In [32]:
corNms

['flag_train',
 'inherited_from_father_Yes',
 'maternal_gene_Yes',
 'respiratory_rate_breathsmin_Tachypnea',
 'heart_rate_ratesmin_Tachycardia',
 'folic_acid_details_periconceptional_Yes',
 'ho_serious_maternal_illness_Yes',
 'assisted_conception_ivfart_Yes',
 'history_of_anomalies_in_previous_pregnancies_Yes',
 'birth_defects_Singular',
 'genes_in_mothers_side_Yes',
 'paternal_gene_Yes',
 'status_Alive',
 'status_Deceased',
 'parental_consent_Yes']

In [34]:
df_names = df.columns.tolist()
df_names.remove('target')
df_names.remove('flag_train')
df_names.remove('myflag')
df_names.remove('genetic_disorder')
df_names.remove('disorder_subclass')
df_names.remove('genetic_disorder_old')
df_names.remove('disorder_subclass_v1')
df_names.remove('genetic_disorder_v1')
df_names.remove('patient_id')


In [ ]:
#df.to_pickle('./processed_if_hbos_pca_cblof_vae_lof_bags1.pkl')

In [36]:
# seperating train and valid after anomaly based features
df_valid = df[df.flag_train==0]
df_train = df[df.flag_train==1]
df_valid= df_valid.drop(['flag_train'],axis=1)
df_train= df_train.drop(['flag_train'],axis=1)

#df_train= df_train.drop_duplicates(subset=['IF_score','target'])
df_valid.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_train = df_train.fillna(-9999)
df_valid = df_valid.fillna(-9999)

In [37]:
df_valid.shape

(9465, 76)

In [38]:
X_score = df_valid.drop(['target'],axis=1) # Test data to be submitted, target data is set to -1

In [39]:
import datetime

x = datetime.datetime.now()
print(x)

2021-08-01 21:31:17.652095


In [40]:
# predict the majority class
def majorityPredictor(arr):    
    lst = arr.tolist()
    occ_= max(lst,key=lst.count)
    return occ_

In [41]:
def getPredictors_multi(selected_classfier,params,df_train,X_score,n_splits,seed_): 
    track_accuracy=[]
    y_score_tot = []   
    folds=StratifiedKFold(n_splits=int(n_splits), shuffle=True, random_state=seed_)
    for train_index, test_index in folds.split(df_train,df_train['target']):
        split_trn, split_val = df_train.iloc[train_index], df_train.iloc[test_index]
        y_train = split_trn['target']
        X_train = split_trn.drop(['target'],axis=1)
        y_valid = split_val['target']
        X_valid = split_val.drop(['target'],axis=1)
        accuracy_1,y_pred_score = myClassfier_multi(selected_classfier,params,X_train, y_train,X_valid,y_valid,X_score)
        track_accuracy.append(accuracy_1)
        y_score_tot.append(y_pred_score)
        
    mean_accuracy_=  np.mean(track_accuracy)
    print("Mean Accuracy: "+str(mean_accuracy_))
    
    final=pd.DataFrame(y_score_tot).T
   
    return final,mean_accuracy_

In [42]:
param_score=[]
## Creating a loss function for Hyper parameter tuning and saving  hyper parameter values and the corresponding accuracy
##  for the defined parameters
def score_multi(params):
    print(params)
    y_pred_score_multi,multi_acc_ = getPredictors_multi(selected_classfier,params,
                                                                                 df_train,X_score,n_splits=5,
                                                                                 seed_=randSeed)
    
    pd.DataFrame([selected_classfier,params,multi_accuracy_,
                  datetime.now().strftime('%Y_%m_%d_%H_%M_%S')]).T.to_csv('hyper_multi.csv',mode='a', header=False)

    return -multi_acc_

In [43]:
from datetime import datetime
datetime.now()


datetime.datetime(2021, 8, 1, 21, 31, 41, 252642)

In [46]:
drop_col = ['myflag', 'genetic_disorder', 'disorder_subclass','genetic_disorder_old', 'disorder_subclass_v1', 'genetic_disorder_v1','patient_id','mytarget']
df_train.drop(columns=drop_col, inplace=True)
X_score.drop(columns=drop_col, inplace=True)

In [47]:
df_train.head()

,patient_age,mothers_age,fathers_age,no._of_previous_abortion,white_blood_cell_count_thousand_per_microliter,blood_cell_count_mcl,test_1,test_2,test_3,test_4,test_5,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,target,inherited_from_father_No,inherited_from_father_Yes,maternal_gene_No,maternal_gene_Yes,respiratory_rate_breathsmin_Normal (30-60),respiratory_rate_breathsmin_Tachypnea,heart_rate_ratesmin_Normal,heart_rate_ratesmin_Tachycardia,gender_Ambiguous,gender_Female,gender_Male,birth_asphyxia_No,birth_asphyxia_Not available,birth_asphyxia_Yes,autopsy_shows_birth_defect_if_applicable_No,autopsy_shows_birth_defect_if_applicable_Not applicable,autopsy_shows_birth_defect_if_applicable_Yes,place_of_birth_Home,place_of_birth_Institute,place_of_birth_Not available,folic_acid_details_periconceptional_No,folic_acid_details_periconceptional_Yes,ho_serious_maternal_illness_No,ho_serious_maternal_illness_Yes,ho_radiation_exposure_xray_No,ho_radiation_exposure_xray_Not applicable,ho_radiation_exposure_xray_Yes,ho_substance_abuse_No,ho_substance_abuse_Not applicable,ho_substance_abuse_Yes,assisted_conception_ivfart_No,assisted_conception_ivfart_Yes,history_of_anomalies_in_previous_pregnancies_No,history_of_anomalies_in_previous_pregnancies_Yes,birth_defects_Multiple,birth_defects_Singular,blood_test_result_abnormal,blood_test_result_inconclusive,blood_test_result_normal,blood_test_result_slightly abnormal,genes_in_mothers_side_No,genes_in_mothers_side_Yes,paternal_gene_No,paternal_gene_Yes,status_Alive,status_Deceased,parental_consent_No,parental_consent_Yes,followup_High,followup_Low,followup_Missing
0,2.0,34.674355,41.665379,2.035378,9.857562,4.760603,0.0,-99.0,-99.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0,True,False,False,True,True,False,True,False,False,False,True,False,True,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,True,False,False,True,True,False,True,False,False,False,False,True,True,False,True,False,False,True,True,False,False
1,4.0,34.363087,23.000000,2.026659,5.522560,4.910669,-99.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1,False,True,True,False,False,True,True,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,False,True,False,True,False,False,True,True,False,False,False,True,False,False,True,True,False,False,True,False,True,True,False,False
2,6.0,41.000000,22.000000,4.000000,7.476777,4.893297,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2,True,False,True,False,True,False,False,True,False,True,False,False,True,False,False,True,False,True,False,False,False,True,True,False,False,False,True,False,True,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,True,False,False,True,False,True,False
3,12.0,21.000000,41.684783,1.000000,7.919321,4.705280,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3,True,False,False,True,False,True,True,False,False,False,True,False,True,False,True,False,False,False,True,False,True,False,False,True,False,False,True,False,True,False,False,True,False,True,False,True,False,True,False,False,False,True,True,False,False,True,False,True,True,False,False
4,11.0,32.000000,41.991621,4.000000,4.098210,4.720703,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,True,False,False,True,False,True,False,True,False,False,True,False,True,False,False,True,False,False,True,False,True,False,False,True,False,True,False,False,True,False,False,True,True,False,True,False,False,True,False,False,False,True,False,True,True,False,True,False,False,True,False


In [48]:
#import hyperopt as hp
from hyperopt import hp
from hyperopt import fmin, tpe
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
import xgboost as xgb
from  lightgbm import LGBMClassifier
import xgboost as XGBClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score, precision_score, log_loss, roc_auc_score, accuracy_score,roc_curve,precision_recall_curve
from datetime import datetime

#### Multi Classifier LGBM model

In [50]:
def myClassfier_multi(selected_classfier,params,X_train, y_train,X_valid,y_valid,X_score): 
      
    X_train = X_train.to_numpy()
    X_valid = X_valid.to_numpy()
    X_score = X_score.to_numpy()
    if(selected_classfier=='xgboost'):
        clf = XGBClassifier(#colsample_bytree =           params[ 'colsample_bytree'], 
                                #colsample_bylevel =      params['colsample_bylevel'],
                                #colsample_bynode =       params['colsample_bynode'],                                                            
                                #gamma =                  params['gamma'],
                                learning_rate =          params['learning_rate'], 
                                max_depth =              int(params[ 'max_depth']), 
                                min_child_weight =       int(params[ 'min_child_weight']), 
                                n_estimators =           int(params[ 'n_estimators']), 
                                nthread =                int(params[ 'nthread']), 
                                objective =              'multi:softmax', 
                                seed =                   params[ 'seed'], 
                                #silent =                 params[ 'silent'], 
                                subsample =              params[ 'subsample'], 
                                verbosity  =             params[ 'verbosity'], 
                                early_stopping_rounds =  int(params[ 'early_stopping_rounds']), 
                                predictor =              params[ 'predictor'], 
                                tree_method =            params[ 'tree_method'],
                                updater =                params['updater'],
                                #num_parallel_tree=50,
                                #n_jobs      =            -1,
                                sampling_method =        params['sampling_method'],
                                #missing =                np.nan,
                                use_label_encoder=False)
        #clf.set_params(**params)
        #print(clf)
    #    predictions = gbm_model.predict(dvalid,
    #                                    ntree_limit=gbm_model.best_iteration + 1)
    #    predictions = (predictions>0.5)
        clf.fit(X_train,y_train,verbose=0)
    elif(selected_classfier =='lightgbm'):
          
        clf = LGBMClassifier(objective              = 'multiclass',
                            #categorical_feature      = cat_cols,
                            #n_estimators             = int(params[ 'n_estimators']),
                            num_leaves               = int(params[ 'num_leaves']),
                            #max_bin = 64,
                            colsample_bytree         = params[ 'colsample_bytree'],
                            learning_rate            = params[ 'learning_rate'],
                            #early_stopping_round    =int( params[ 'early_stopping_round']),
                            #max_depth                = 30,
                            min_child_samples        = int(params[ 'min_child_samples']) ,
                            #lambda_l1                = params['lambda_l1'],
                            #lambda_l2                = params['lambda_l2'],
                            subsample                = params['subsample'],
                            bagging_freq             = int(params['bagging_freq']),
                            verbosity                = -1,
                            device                   = params[ 'device'],
                            first_metric_only        = params[ 'first_metric_only'],
                            #eval_metric              = params[ 'eval_metric'],
                            #metric                   = params[ 'metric'],
                            n_jobs                   = int(params[ 'n_jobs']),
                            #eval_names               = params[ 'eval_names'],
                            random_state             = int(params[ 'random_state']),
                            tree_learner            = params[ 'tree_learner']) 
        
        clf.fit(X_train,y_train,verbose=0) 
    elif(selected_classfier =='RF'):
          
        clf = RandomForestClassifier( 
                            n_estimators              = int(params['n_estimators']),
                            criterion                 = params['criterion'],
                            max_depth                 = int(params['max_depth']),
                            min_samples_split         = int(params['min_samples_split']),            
                            min_samples_leaf          = int(params['min_samples_leaf']),            
                            min_weight_fraction_leaf  = params['min_weight_fraction_leaf'],     
                            max_features              = params['max_features'],    
                            min_impurity_decrease     = params['min_impurity_decrease'],     
                            oob_score                 = params['oob_score'],
                            max_samples               = params['max_samples']
                            )

        clf.fit(X_train,y_train)
        
    y_pred= clf.predict(X_valid)
    y_score_pred = clf.predict(X_score)
    accuracy_1=accuracy_score(y_valid,y_pred)
    del clf
    gc.collect()
            
            
    return accuracy_1,y_score_pred

In [51]:
## Function to create training and validation data based on Stratified Kfold Strategy.
## Outputs the predictions on test data and mean training accuracy score across K folds
def getPredictors_multi(selected_classfier,params,df_train,X_score,n_splits,seed_): 
    track_accuracy=[]
    y_score_tot = []   
    folds=StratifiedKFold(n_splits=int(n_splits), shuffle=True, random_state=seed_)
    for train_index, test_index in folds.split(df_train,df_train['target']):
        split_trn, split_val = df_train.iloc[train_index], df_train.iloc[test_index]
        y_train = split_trn['target']
        X_train = split_trn.drop(['target'],axis=1)
        y_valid = split_val['target']
        X_valid = split_val.drop(['target'],axis=1)
        accuracy_1,y_pred_score = myClassfier_multi(selected_classfier,params,X_train, y_train,X_valid,y_valid,X_score)
        track_accuracy.append(accuracy_1)
        y_score_tot.append(y_pred_score)
        
    mean_accuracy_=  np.mean(track_accuracy)
    print("Mean Accuracy: "+str(mean_accuracy_))
    
    final=pd.DataFrame(y_score_tot).T
   
    return final,mean_accuracy_

In [52]:
param_score=[]
## Creating a loss function for Hyper parameter tuning and saving  hyper parameter values and the corresponding accuracy
##  for the defined parameters
def score_multi(params):
    print(params)
    y_pred_score_multi,multi_acc_ = getPredictors_multi(selected_classfier,params,
                                                                                 df_train,X_score,n_splits=5,
                                                                                 seed_=randSeed)
    
    pd.DataFrame([selected_classfier,params,multi_acc_,
                  datetime.now().strftime('%Y_%m_%d_%H_%M_%S')]).T.to_csv('hyper_multi.csv',mode='a', header=False)

    return -multi_acc_

In [54]:
## Code for hyper paramter tuning  using Light GBM as the classifier and 1000 evaluations
selected_classfier='lightgbm'
space = parameterSpace(selected_classfier)
       
    # Use the fmin function from Hyperopt to find the best hyperparameters
best_lightgbm = fmin(score_multi, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=50,rstate= np.random.RandomState(randSeed))

{'bagging_freq': 15.0, 'colsample_bytree': 0.765, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 1.2785178546988023, 'lambda_l2': 24.28210493447922, 'learning_rate': 0.17500000000000002, 'min_child_samples': 44.0, 'n_jobs': -1, 'num_leaves': 340.0, 'random_state': 17, 'subsample': 0.825, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will b

bagging_freq is set=38, subsample_freq=0 will be ignored. Current value: bagging_freq=38                               
Mean Accuracy: 0.3518453427065027                                                                                      
{'bagging_freq': 17.0, 'colsample_bytree': 0.255, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 1.484935638644304, 'lambda_l2': 0.20758511163197, 'learning_rate': 0.17500000000000002, 'min_child_samples': 12.0, 'n_jobs': -1, 'num_leaves': 540.0, 'random_state': 17, 'subsample': 0.87, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be i

bagging_freq is set=21, subsample_freq=0 will be ignored. Current value: bagging_freq=21                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=21, subsample_freq=0 will be ignored. Current value: bagging_freq=21                               
Mean Accuracy: 0.3675119256841577                                                                                      
{'bagging_freq': 23.0, 'colsample_bytree': 0.21, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 142.2037375347965, 'lambda_l2': 43.015645618287195, 'learning_rate': 0.18, 'min_child_samples': 16.0, 'n_jobs': -1, 'num_leaves': 110.0, 'random_state': 17, 'subsample': 0.21, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=23, subsample_freq=0 will be ignored. Curren

bagging_freq is set=31, subsample_freq=0 will be ignored. Current value: bagging_freq=31                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=31, subsample_freq=0 will be ignored. Current value: bagging_freq=31                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=31, subsample_freq=0 will be ignored. Current value: bagging_freq=31                               
Mean Accuracy: 0.3354255586241527                                                                                      
{'bagging_freq': 21.0, 'colsample_bytree': 0.63, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 0.014497072794450847, 'lambda_l2': 0.6004778465514929, 'learning_rate': 0.195, 'min_child_samples': 26.0, 'n_jobs': -1, 'num_leaves': 340.0, 'rand

bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
Mean Accuracy: 0.3651518955561135                                                                                      
{'bagging_freq': 49.0, 'colsample_bytree

...

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=19, subsample_freq=0 will be ignored. Current value: bagging_freq=19                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=19, subsample_freq=0 will be ignored. Current value: bagging_freq=19                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=19, subsample_freq=0 will be ignored. Current value: bagging_freq=19                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=19, subsample_freq=0 will be ignored. Current value: bagging_freq=19                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                    

In [55]:
best_lightgbm

{'bagging_freq': 4.0,
 'colsample_bytree': 0.705,
 'lambda_l1': 0.9687039737275855,
 'lambda_l2': 0.04309002126693597,
 'learning_rate': 0.02,
 'min_child_samples': 54.0,
 'num_leaves': 30.0,
 'subsample': 0.69}

In [56]:
lgbm_params={'bagging_freq': 4.0, 'colsample_bytree': 0.705, 'device': 'cpu', 'eval_names': 'tanzu',
             'first_metric_only': False, 'lambda_l1': 0.9687039737275855, 'lambda_l2': 0.04309002126693597, 
             'learning_rate': 0.02, 'max_depth': -1, 'min_child_samples': 54.0, 'n_jobs': -1, 'num_leaves': 30.0,
             'objective': 'multiclass', 'random_state': 17, 'subsample': 0.69, 'tree_learner': 'serial'}
selected_classfier='lightgbm'
y_pred_score_multi,mean_accuracy_= getPredictors_multi(selected_classfier,lgbm_params,df_train,X_score,
                                                        n_splits=5,seed_=randSeed)

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Mean Accuracy: 0.3926688425809691


In [57]:
y_score_multi_pred= y_pred_score_multi.astype(int).apply(majorityPredictor,axis=1) 

In [58]:
y_score_multi_pred.value_counts()

1    3118
3    2930
7    1673
2    1142
5     277
0     218
6      78
8      29
dtype: int64

In [59]:
multi_class_df=pd.DataFrame()
multi_class_df['pred_lgbm'] = y_score_multi_pred
multi_class_df['patient_id'] = df_valid.patient_id.values
multi_class_df['lgbm_class'] = multi_class_df['pred_lgbm'].apply(lambda x:class_decoding[x])
multi_class_df.to_csv('multi_df_lgbm_combining.csv',index=False)

#### XGBoost Undersampling

In [ ]:
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler,BorderlineSMOTE,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import pickle

In [ ]:
df_train['target'].value_counts()

In [ ]:
sampling_str1 = {0: 2260, 1: 2260, 2: 2260}
sampling_str2 = {0: 2000, 1: 2000, 2: 2000}
sampling_str3 = {0: 5000, 1: 6000, 2: 2260}
sampling_str4 = {0: 2260, 1: 2260, 2: 2260}
sampling_str5 = {0: 2260, 1: 4000, 2: 2000}
sampling_str6 = {0: 1000, 1: 500, 2: 100}
sampling_str7 = {0: 1117, 1: 837, 2: 300}

In [ ]:
rfNms = df_train.columns.to_list()
rfNms.remove('target')

In [ ]:
import xgboost as xgb
def xgb_train_on_sampling_one_fold(training,sampling,i,valid=None):
    
    results = []
    myTgt = 'target'

    under2 = RandomUnderSampler(sampling_strategy=sampling,random_state=42)
    over = RandomOverSampler(random_state=42)
    pipe = Pipeline(steps=[('u', under2),('o', over)])

    X_resampled, y_resampled = pipe.fit_resample(training[rfNms], training[myTgt])

    clf = xgb.XGBClassifier( 
            objective='multi:softprob',
             learning_rate = 0.02,
           subsample = 0.35,
           colsample_bytree = 0.7,
           min_child_weight = 40,
           gamma = 10,
#         n_estimators= 2500,
           max_depth = 3)
        
    h = clf.fit(X_resampled, y_resampled)
    pickle.dump(clf, open(f'xgb_anomality_model1_{i}.pkl', 'wb'))

In [ ]:

sampling_list = [sampling_str1,sampling_str2,sampling_str3,sampling_str4,sampling_str5,sampling_str6,sampling_str7]
for i,d in enumerate(sampling_list):
    xgb_train_on_sampling_one_fold(df_train,sampling=d,i=i)

In [ ]:
def xgb_evaluate(folds):
    preds = []
        
    for fold in range(folds):
        model_xgb = pickle.load(open(f'xgb_anomality_model1_{fold}.pkl', 'rb'))
        preds.append(model_xgb.predict_proba(X_score[rfNms]))
    return np.asarray(preds)
xgb_preds = xgb_evaluate(4)  

In [ ]:
from scipy.stats import hmean

preds = hmean(xgb_preds,axis=0)
preds = preds.argmax(axis=1)
len(preds)

In [ ]:
#multi_class_df=pd.DataFrame()
multi_class_df['xgb_us_pred'] = preds
multi_class_df['xgb_us_class'] = multi_class_df['xgb_us_pred'].apply(lambda x:class_decoding[x])
multi_class_df['lgbm_class'] = multi_class_df['class_pred_lgbm'].apply(lambda x:class_decoding[x])
multi_class_df.to_csv('multi_df_final.csv',index=False)

In [ ]:
multi_class_df.head()

##### Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler 

In [ ]:
## Train and hold out sample
myTgt = 'target'
y1 = df_train[myTgt]
X_train_f, X_valid_f, y_train, y_valid = train_test_split(df_train[rfNms], y1, test_size=0.10, random_state=12345, stratify=y1)
print(X_train_f.shape)
print(X_valid_f.shape)
X_train_f.reset_index(drop=True, inplace=True)
X_valid_f.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_valid.reset_index(drop=True, inplace=True)


In [ ]:
space = {
    "n_estimators": hp.choice("n_estimators", [100, 250, 500,600,900,1000,1200]),
    #"n_estimators": hp.quniform("n_estimators", 50, 1500, 20), #number of trees, change it to 1000 for better results
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
    #"min_samples_split": hp.quniform("min_samples_split", 2, 200,2)
}

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_f)

In [ ]:
def hyperparameter_tuning(params):
    clf = RandomForestClassifier(**params,n_jobs=-1)
    acc = cross_val_score(clf, X_scaled, y_train,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
# Initialize trials object
from hyperopt import Trials,STATUS_OK
trials = Trials()
best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=10,
    #trials=trials
)

print("Best: {}".format(best))

In [ ]:
best

In [ ]:
%%time 
pred____ = pd.DataFrame()
pred = pd.DataFrame()
i=0

seed=42
clf =RandomForestClassifier(n_estimators= 1000, max_features='auto', max_depth= 11, class_weight= 'balanced', criterion='entropy', random_state=seed)  #Tuned parameters
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)  #Making 2 splits to avoid overfitting when predicting for unseen data

cv_score_rf_b =[]
cv_score_rf_f = []

for train_index,test_index in kf.split(X_train_f,y_train.values):
    i+=1
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X_train_f.loc[train_index],X_train_f.loc[test_index]
    ytr,yvl = y_train.loc[train_index],y_train.loc[test_index]
    
    #model
    clf.fit(xtr,ytr)

    pred[i] = clf.predict(X_valid_f)

    print('Balanced Accuracy on validation set:', balanced_accuracy_score(y_valid,pred[i]))
    cv_score_rf_b.append(balanced_accuracy_score(y_valid,pred[i]))

    print('F1 Score on validation set:',f1_score(y_valid,pred[i],average='macro'))
    cv_score_rf_f.append(f1_score(y_valid,pred[i],average='macro'))  

    pred____[i] = clf.predict(X_score[rfNms])  #prediction on the eval dataset
    

In [ ]:
pred['Final_rf'] = pred.mode(axis=1).iloc[:, 0] 

In [ ]:
#Plot classfication report for RF
from sklearn.metrics import classification_report
print(classification_report(y_valid,pred['Final_rf']))

In [ ]:
pred____['Final_rf'] = pred____.mode(axis=1).iloc[:, 0] 
pred____.head()

In [ ]:
#multi_class_df=pd.DataFrame()
multi_class_df['rf_us_pred'] = pred____['Final_rf']
multi_class_df['rf_us_class'] = multi_class_df['rf_us_pred'].apply(lambda x:class_decoding[x])
multi_class_df.to_csv('multi_df_final.csv',index=False)

In [ ]:
multi_class_df.rf_us_class.value_counts()

In [ ]:
multi_class_df.xgb_us_class.value_counts()

In [ ]:
multi_class_df.lgbm_class.value_counts()

In [ ]:
multi_class_df.head()

In [ ]:
multi_class_df['pred_multi']=multi_class_df[['xgb_us_class','lgbm_class','rf_us_class']].apply(majorityPredictor,axis=1)

In [ ]:
multi_class_df.pred_multi.value_counts()

In [ ]:
multi_class_df.to_csv("pred_genetic_disorder_phase1.csv",index=False)